# Modelo do transporte - parte 3

## Importação da biblioteca

In [1]:
import pyomo.environ as pyo

## Modelo computacional

In [2]:
modelo = pyo.AbstractModel()

### Conjuntos

$I \colon \text{Conjunto de distribuidores,} \; I = \{1,2,\ldots,m\},$

$J \colon \text{Conjunto de consumidores,} \; J = \{1,2,\ldots,n\}.$

### Parâmetros

$c_{ij} \colon \text{Custo unitário de transporte do distribuidor }i \in I \text{ para o consumidor }j \in J,$

$a_i \colon \text{Capacidade de fornecimento do distribuidor }i \in I$,

$b_j \colon \text{Demanda do consumidor }j \in J.$

### Variáveis de decisão

$x_{ij} \colon \text{Quantidade a ser transportada do distribuidor }i \in I \text{ para o consumidor }j \in J.$

### Função objetivo

$\text{min }z(x) = \sum_\limits{i \in I} \sum_\limits{j \in J} c_{ij} x_{ij}.$

### Restrições

#### Capacidade

$\sum_\limits{j \in J} x_{ij} = a_i, \;\; \forall i \in I,$

#### Demanda

$\sum_\limits{i \in I} x_{ij} = b_j, \;\; \forall j \in J,$

#### Não negatividade

$x_{ij} \geq 0 \;\; \forall i \in I, j \in J.$

In [3]:
# Parâmetros auxiliares:
modelo.m = pyo.Param()
modelo.n = pyo.Param()
modelo.custos = pyo.Param(within=pyo.Any)
modelo.capacidade = pyo.Param(within=pyo.Any)
modelo.demanda = pyo.Param(within=pyo.Any)

# Conjuntos:
modelo.I = pyo.RangeSet(modelo.m)
modelo.J = pyo.RangeSet(modelo.n)

# Parâmetros:
modelo.c = pyo.Param(modelo.I, modelo.J, initialize=lambda modelo, i, j: modelo.custos()[i-1][j-1], mutable=True)
modelo.a = pyo.Param(modelo.I, initialize=lambda modelo, i: modelo.capacidade()[i-1])
modelo.b = pyo.Param(modelo.J, initialize=lambda modelo, j: modelo.demanda()[j-1])

# Variáveis de decisão:
modelo.x = pyo.Var(modelo.I, modelo.J, within=pyo.NonNegativeReals)

# Função objetivo:
def regra_z(mod):
    return pyo.summation(mod.c, mod.x)

modelo.z = pyo.Objective(rule=regra_z, sense=pyo.minimize) # minimize = default

# Restrições de capacidade:
def regra_capacidade(mod, i):
    return sum(mod.x[i,j] for j in mod.J) <= mod.a[i]

modelo.restr_capacidade = pyo.Constraint(modelo.I, rule=regra_capacidade)

# Restrições de demanda:
def regra_demanda(mod, j):
    return sum(mod.x[i,j] for i in mod.I) >= mod.b[j]

modelo.restr_demanda = pyo.Constraint(modelo.J, rule=regra_demanda)

# Lembrete: as restrições de não negatividade já foram definidas através do argumento 'within' das variáveis de decisão

## Função para lidar com a classe data_portal

In [4]:
def converter_dict_para_data_portal(params):
    data_portal = pyo.DataPortal()
    
    def __padronizar_parametros(param):
        if isinstance(param, list):
            if isinstance(param[0], list):
                param = [tuple(row) for row in param]      
        return {None: param}
    
    for key in params:
        data_portal[key] = __padronizar_parametros(params[key])
        
    return data_portal

## Instância 1

### Leitura dos dados de entrada

In [5]:
import pandas as pd

In [6]:
dados_entrada = 'dados_entrada.xlsx'

In [7]:
df_custos = pd.read_excel(open(dados_entrada, 'rb'), sheet_name='custos') 

custos = df_custos[['origem_id', 'destino_id', 'custo_unitario']].pivot(
    index='origem_id', 
    columns='destino_id', 
    values='custo_unitario').reset_index(drop=True).to_numpy()

In [8]:
df_fornecedores = pd.read_excel(open(dados_entrada, 'rb'), sheet_name='fornecedores')

capacidade = df_fornecedores['capacidade'].to_numpy()

In [10]:
df_consumidores = pd.read_excel(open(dados_entrada, 'rb'), sheet_name='consumidores')

demanda = df_consumidores['demanda'].to_numpy()

In [11]:
m = len(capacidade)
n = len(demanda)

### Consolidação dos dados de entrada em um dicionário

In [12]:
dict_params = {}
dict_params['custos'] = custos
dict_params['capacidade'] = capacidade
dict_params['demanda'] = demanda
dict_params['m'] = m
dict_params['n'] = n

In [13]:
dict_params

{'custos': array([[12, 22, 30],
        [18, 24, 32],
        [22, 15, 34]]),
 'capacidade': array([100, 140, 160]),
 'demanda': array([120, 130, 150]),
 'm': 3,
 'n': 3}

### Transformação do dicionário de parâmetros em data_portal

In [14]:
params_dp = converter_dict_para_data_portal(dict_params)

In [15]:
type(params_dp)

pyomo.dataportal.DataPortal.DataPortal

### Criação da instância do modelo

In [16]:
instancia = modelo.create_instance(params_dp) 

### Resolve a instância

In [17]:
resultado = pyo.SolverFactory('glpk').solve(instancia)

In [18]:
resultado.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 8370.0
  Upper bound: 8370.0
  Number of objectives: 1
  Number of constraints: 7
  Number of variables: 10
  Number of nonzeros: 19
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.020292043685913086
# ----------------------------------------------------------
#   Solution Information
# --------------------------------

In [19]:
instancia.x.pprint()

x : Size=9, Index=x_index
    Key    : Lower : Value : Upper : Fixed : Stale : Domain
    (1, 1) :     0 : 100.0 :  None : False : False : NonNegativeReals
    (1, 2) :     0 :   0.0 :  None : False : False : NonNegativeReals
    (1, 3) :     0 :   0.0 :  None : False : False : NonNegativeReals
    (2, 1) :     0 :  20.0 :  None : False : False : NonNegativeReals
    (2, 2) :     0 :   0.0 :  None : False : False : NonNegativeReals
    (2, 3) :     0 : 120.0 :  None : False : False : NonNegativeReals
    (3, 1) :     0 :   0.0 :  None : False : False : NonNegativeReals
    (3, 2) :     0 : 130.0 :  None : False : False : NonNegativeReals
    (3, 3) :     0 :  30.0 :  None : False : False : NonNegativeReals


In [20]:
instancia.z()

8370.0

### Impressão dos resultados

In [21]:
for j in instancia.J:
    consumidor = df_consumidores['consumidor_descr'].iloc[j-1]
    print(f'* Consumidor {consumidor} receberá:')
    c_total = 0
    for i in instancia.I:
        fornecedor = df_fornecedores['fornecedor_descr'].iloc[i-1]
        x = instancia.x[i,j]()
        c = instancia.c[i,j]()*x
        c_total = c_total + c
        if x > 0:
            print(f'     - {x} unidades de {fornecedor} - Valor: R$ {c}')
    print(f'     - Total: R$ {c_total}')
    print('')
print(f'Custo total de transporte: R$ {instancia.z()}')

* Consumidor São Paulo receberá:
     - 100.0 unidades de Osasco - Valor: R$ 1200.0
     - 20.0 unidades de Sorocaba - Valor: R$ 360.0
     - Total: R$ 1560.0

* Consumidor Rio de Janeiro receberá:
     - 130.0 unidades de São Sebastião - Valor: R$ 1950.0
     - Total: R$ 1950.0

* Consumidor Curitiba receberá:
     - 120.0 unidades de Sorocaba - Valor: R$ 3840.0
     - 30.0 unidades de São Sebastião - Valor: R$ 1020.0
     - Total: R$ 4860.0

Custo total de transporte: R$ 8370.0


## Instância 2

In [22]:
dados_entrada = 'dados_entrada2.xlsx'

In [24]:
df_custos = pd.read_excel(open(dados_entrada, 'rb'), sheet_name='custos') 
custos = df_custos[['origem_id', 'destino_id', 'custo_unitario']].pivot(
    index='origem_id', 
    columns='destino_id', 
    values='custo_unitario').reset_index(drop=True).to_numpy()


df_fornecedores = pd.read_excel(open(dados_entrada, 'rb'), sheet_name='fornecedores')
capacidade = df_fornecedores['capacidade'].to_numpy()


df_consumidores = pd.read_excel(open(dados_entrada, 'rb'), sheet_name='consumidores')
demanda = df_consumidores['demanda'].to_numpy()


m = len(capacidade)
n = len(demanda)

In [25]:
dict_params = {}
dict_params['custos'] = custos
dict_params['capacidade'] = capacidade
dict_params['demanda'] = demanda
dict_params['m'] = m
dict_params['n'] = n

params_dp = converter_dict_para_data_portal(dict_params)

In [26]:
instancia2 = modelo.create_instance(params_dp)

In [27]:
resultado = pyo.SolverFactory('glpk').solve(instancia2)

In [28]:
resultado.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 7240.0
  Upper bound: 7240.0
  Number of objectives: 1
  Number of constraints: 8
  Number of variables: 13
  Number of nonzeros: 25
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.0334019660949707
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------

In [31]:
instancia.x.pprint()

x : Size=9, Index=x_index
    Key    : Lower : Value : Upper : Fixed : Stale : Domain
    (1, 1) :     0 : 100.0 :  None : False : False : NonNegativeReals
    (1, 2) :     0 :   0.0 :  None : False : False : NonNegativeReals
    (1, 3) :     0 :   0.0 :  None : False : False : NonNegativeReals
    (2, 1) :     0 :  20.0 :  None : False : False : NonNegativeReals
    (2, 2) :     0 :   0.0 :  None : False : False : NonNegativeReals
    (2, 3) :     0 : 120.0 :  None : False : False : NonNegativeReals
    (3, 1) :     0 :   0.0 :  None : False : False : NonNegativeReals
    (3, 2) :     0 : 130.0 :  None : False : False : NonNegativeReals
    (3, 3) :     0 :  30.0 :  None : False : False : NonNegativeReals


In [30]:
instancia2.x.pprint()

x : Size=12, Index=x_index
    Key    : Lower : Value : Upper : Fixed : Stale : Domain
    (1, 1) :     0 :  90.0 :  None : False : False : NonNegativeReals
    (1, 2) :     0 :   0.0 :  None : False : False : NonNegativeReals
    (1, 3) :     0 :   0.0 :  None : False : False : NonNegativeReals
    (2, 1) :     0 :  30.0 :  None : False : False : NonNegativeReals
    (2, 2) :     0 :  30.0 :  None : False : False : NonNegativeReals
    (2, 3) :     0 :  50.0 :  None : False : False : NonNegativeReals
    (3, 1) :     0 :   0.0 :  None : False : False : NonNegativeReals
    (3, 2) :     0 : 100.0 :  None : False : False : NonNegativeReals
    (3, 3) :     0 :   0.0 :  None : False : False : NonNegativeReals
    (4, 1) :     0 :   0.0 :  None : False : False : NonNegativeReals
    (4, 2) :     0 :   0.0 :  None : False : False : NonNegativeReals
    (4, 3) :     0 : 100.0 :  None : False : False : NonNegativeReals


### Impressão dos resultados

In [33]:
for j in instancia2.J:
    consumidor = df_consumidores['consumidor_descr'].iloc[j-1]
    print(f'* Consumidor {consumidor} receberá:')
    c_total = 0
    for i in instancia2.I:
        fornecedor = df_fornecedores['fornecedor_descr'].iloc[i-1]
        x = instancia2.x[i,j]()
        c = instancia2.c[i,j]()*x
        c_total = c_total + c
        if x > 0:
            print(f'     - {x} unidades de {fornecedor} - Valor: R$ {c}')
    print(f'     - Total: R$ {c_total}')
    print('')
print(f'Custo total de transporte: R$ {instancia2.z()}')

* Consumidor São Paulo receberá:
     - 90.0 unidades de Osasco - Valor: R$ 1080.0
     - 30.0 unidades de Sorocaba - Valor: R$ 540.0
     - Total: R$ 1620.0

* Consumidor Rio de Janeiro receberá:
     - 30.0 unidades de Sorocaba - Valor: R$ 720.0
     - 100.0 unidades de São Sebastião - Valor: R$ 1500.0
     - Total: R$ 2220.0

* Consumidor Curitiba receberá:
     - 50.0 unidades de Sorocaba - Valor: R$ 1600.0
     - 100.0 unidades de Santos - Valor: R$ 1800.0
     - Total: R$ 3400.0

Custo total de transporte: R$ 7240.0
